In [ ]:
!pip install pycirclize pandas

In [ ]:
%matplotlib inline
from pycirclize import Circos
from pycirclize.parser import Genbank
from matplotlib.patches import Patch
from matplotlib import pyplot as plt

import pandas as pd

import re

In [ ]:
class MyRadius:
    def __init__(self):
        self._radius = 100

    def get(self, thin=False):
        width = 4 if thin else 8
        value = self._radius - width, self._radius
        self._radius -= width
        return value

def get_name(old):
    return '-'.join(old.split('-')[1:])


def get_name2(old):
    return old.split('-')[0]


def percent_to_float(old: str) -> float:
    return float(old.rstrip('%')) / 100

In [ ]:
# read data
# evaluation result table, remove abnormal genes
count_threshold = 10
filename = r'rodents_Evaluation.csv'
gb_file = r'extend.gb'
gb = Genbank(gb_file)
data_raw = pd.read_csv(filename)
data_raw['Name'] = data_raw.Loci.apply(get_name)
data_raw['Loci'] = data_raw.Loci.apply(get_name2)
data_raw['Gap_Ratio'] = data_raw['Gap_Ratio'].apply(percent_to_float)
data_raw['Tree_Res'] = data_raw['Tree_Res'].apply(percent_to_float)
data_raw['Total_GC'] = data_raw['Total_GC'].apply(percent_to_float)
data_raw['Observed_Res'] = data_raw['Observed_Res'].apply(percent_to_float)
data_raw2 = data_raw[data_raw.Samples >= count_threshold]
data = data_raw2[data_raw2.Loci != 'gene']
data_names_raw = data.Name.tolist()
data_names = list()
for i in data_names_raw:
    new = re.sub('trn(.|fM)_...', r'trn\1', i)
    data_names.append(new)


In [ ]:
# reference organelle genome genbank file from OGU.gb2fasta
# total length - ir length
r = MyRadius()
# yellow,yellow/gray,blue,blue,green?
feature_colors = '#FFD93D,#DDDDDD,#4D96FF,#9D96FF,#6BCB77'.split(',')
track_colors = list(
    reversed('#F7931B,#66B3FF,#338CFF,#3F51B5,#FB9883,#CC5500,#FF6347,#36454F,#778877'.split(',')))
features = ('CDS', 'intron', 'tRNA', 'rRNA', 'spacer')
colname_text = (('Tree_Res', 'Tree resolution'), ('PD', 'PD'), ('PD_stem', 'PD-stem'),
                ('PD_terminal', 'PD-terminal'),
                ('Observed_Res', 'Observed resolution'), ('Pi', 'Pi'), ('Entropy', 'Shannon index'),
                ('Total_GC', 'GC ratio'),
                ('Gap_Ratio', 'Gap ratio'))


In [ ]:
fig = plt.figure(figsize=(12, 12))
# circos = Circos(sectors, space=0.5)
# genome gb file as template, no extra treatmetn
circos = Circos(sectors={gb.name: gb.range_size},
                sector2clockwise={gb.name: False})
sector = circos.sectors[0]
r1 = r.get()
feature_track = sector.add_track(r1)
feature_track.axis(fc="#FFFFFF", ec="none")
gene_labels = []
gene_list = []
last_gene = ''
gene_set = set()
ir_pos_list = set()
for feat in gb.extract_features("gene"):
    start, end = int(str(feat.location.start)), int(str(feat.location.end))
    pos = (start + end) / 2
    label = feat.qualifiers.get("gene", ["??"])[0]
    if label == "" or label.startswith("hypothetical") or label == last_gene:
        continue
    if len(label) > 20:
        label = label[:20] + "..."
    last_gene = label
    gene_list.append(pos)
    gene_labels.append(label)
    if label in gene_set:
        ir_pos_list.add(pos)
    gene_set.add(label)

# Plot CDS product labels on outer position
feature_track.xticks(
    gene_list,
    gene_labels,
    label_orientation="vertical",
    show_bottom_line=True,
    label_size=6,
    #line_kws=dict(ec="white"),
)
# extract names
pos_list, labels = [], []
last_name = ''
for feature, color in zip(features, feature_colors):
    feature_track.genomic_features(
        gb.extract_features(feature),
        plotstyle="box",
        r_lim=r1,
        fc=color, alpha=1, lw=0.1, ec='black')
    for f in gb.extract_features(feature):
        start, end = int(str(f.location.start)), int(str(f.location.end))
        pos = end
        if feature == 'intron':
            label = f.qualifiers['gene'][0] + '.' + f.qualifiers['number'][0]
        elif feature != 'spacer':
            label = f.qualifiers.get("gene", ["??"])[0]
            if label == last_name:
                continue
        else:
            label = f.qualifiers['upstream'][0] + '-' + f.qualifiers['downstream'][0]
        last_name = label
        pos_list.append(pos)
        labels.append(label)
feature_track.genomic_features(gb.extract_features('D-loop'), plotstyle='box', r_lim=r1,
                              fc='#999999', alpha=1, lw=0.2, ec='black')
labels_set = set(labels)
data_names_set = set(data_names)
#a_b = sorted(list(data_names_set-labels_set))
#print('data_names_set-labels_set', a_b, len(a_b))
#print('\n'*3)
#b_a = sorted(list(labels_set-data_names_set))
#print('labels_set-data_names_set', b_a, len(b_a))
intersection = data_names_set & labels_set
clean_data = data.query('Name in @intersection')
label_pos = list()
label_pos_dict = dict()
ir_second_label_pos = list()
for key, value in zip(labels, pos_list):
    if key not in intersection:
        continue
    label_pos.append((key, value))
    label_pos_dict[key] = value

clean_data['Position'] = [label_pos_dict[x] for x in clean_data.Name]
clean_data = clean_data.sort_values(by=['Position'])
clean_pos = clean_data.Position.tolist()
widths = list()
widths.append(clean_pos[0])
for i in range(1, len(clean_pos)):
    w = clean_pos[i] - clean_pos[i - 1]
    widths.append(w)

#print(clean_data[['Name','Position','Loci']].head, clean_data.shape, len(clean_pos))
#print(clean_data.Name.tolist(), clean_pos)
#print(ir_second_label_pos)
def draw_bar(track, x_, y_, w_, text):
    max_y = max(y_)
    color = track_colors.pop()
    for x, y, w in zip(x_, y_, w_):
        # bar need x[list], y[list]
        # x should adjusted by width
        track.bar([x - w / 2], [y], width=w, color=color, 
                  vmin=0, vmax=max_y)

    # ec='black', lw=0.5


for col, text in colname_text:
    track = sector.add_track(r.get(), r_pad_ratio=0.1)
    track.axis()
    col_data = clean_data[col].tolist()
    draw_bar(track, clean_pos, col_data, widths, text)

# print(clean_data.columns)

fig = circos.plotfig()

track_colors2 = '#F7931B,#66B3FF,#338CFF,#3F51B5,#FB9883,#CC5500,#FF6347,#36454F,#778877'.split(',')
rect_handles = []
for name, color in zip(features, feature_colors):
    rect_handles.append(Patch(color=color, label=name))
rect_handles.append(Patch(color='#999999', label='D-loop'))
for name, color in zip(colname_text, track_colors2):
    rect_handles.append(Patch(color=color, label=name[1]))
_ = circos.ax.legend(handles=rect_handles, bbox_to_anchor=(1, 0),
                     loc='best', fontsize=8, title="Types", ncol=3)


In [ ]:
fig.savefig('out.pdf')

